In [2]:
import pandas as pd
import re
import igraph as ig
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import os  
from sys import argv

In [ ]:
#For old twitter data
if len(argv) < 1:
    raise ValueError('Hashtag must be provided as argument')
hashtag = argv[1]

data = pd.read_csv('../data/twitter/talk.csv')

#create edgelist to create a graph with actor.preferredUsername connecting to the user_mentions of each tweet
edgelist = []
for i in range(len(data)):
    if i % 10000 == 0:
        print(i, len(data))
        print(len(edgelist))
    if f"#{hashtag} " in data.iloc[i]['body']:
        user = data.iloc[i]['actor.preferredUsername']
        mentions = data.iloc[i]['twitter_entities.user_mentions']
        if mentions != '[]':
            mentions = re.findall(r'"screen_name"\s*:\s*"([^"]+)"', mentions)
            for mention in mentions:
                edgelist.append((user, mention))

dir_path = f'../data/twitter'
file_path = os.path.join(dir_path, f'{hashtag}.txt')

with open(file_path, 'a', encoding='utf-8') as f:
    for edge in edgelist:
        f.write(f"{edge}\n")

New Twitter data

In [3]:
filename = 'election'

data = pd.read_pickle(f'../data/twitter/{filename}_tweets')

In [4]:
data[0]

{'created_at': 'Tue Jul 16 17:47:33 +0000 2019',
 'id': 1151186622597255168,
 'id_str': '1151186622597255168',
 'text': "@GOP Looks like there's some rebranding going on for #Election2020.\n\n#WestVirginia #wvpol #2020Elections \n\nhttps://t.co/hG4gWXWo8q",
 'truncated': False,
 'entities': {'hashtags': [{'text': 'Election2020', 'indices': [53, 66]},
   {'text': 'WestVirginia', 'indices': [69, 82]},
   {'text': 'wvpol', 'indices': [83, 89]},
   {'text': '2020Elections', 'indices': [90, 104]}],
  'symbols': [],
  'user_mentions': [{'screen_name': 'GOP',
    'name': 'GOP',
    'id': 11134252,
    'id_str': '11134252',
    'indices': [0, 4]}],
  'urls': [{'url': 'https://t.co/hG4gWXWo8q',
    'expanded_url': 'https://jacobinmag.com/2019/07/west-virginia-stephen-smith-governor-2020-election/',
    'display_url': 'jacobinmag.com/2019/07/west-v…',
    'indices': [107, 130]}]},
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'in_reply_to_status_id': None

In [5]:
# Convert the list to a DataFrame
df = pd.DataFrame(data)

# Count how many have a in_reply_to_user_id and not
print(df['in_reply_to_user_id'].isna().sum())
print(df['in_reply_to_user_id'].notna().sum())

49159
2266


In [6]:
#create edgelist using in_reply_to_user_id and user id
edgelist = []
for i in range(len(df)):
    edgelist.append((df.iloc[i]['user']['id'], df.iloc[i]['in_reply_to_user_id']))
#remove nans
edgelist = [edge for edge in edgelist if str(edge[1]) != 'nan']
#make int
edgelist = [(int(edge[0]), int(edge[1])) for edge in edgelist]

In [7]:
#save edgelist to file
dir_path = f'../data/twitter'
file_path = os.path.join(dir_path, f'{filename}_edges.txt')

with open(file_path, 'a', encoding='utf-8') as f:
    for edge in edgelist:
        f.write(f"{edge[0]},{edge[1]}\n")
